In [2]:
import os, re
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import math
from math import log
from tqdm import tqdm_notebook as tqdm
from scipy import spatial
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
import json
import scipy.sparse

In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [4]:
def get_dls(corpus):
    dls = []
    
    for text in tqdm(corpus):
        dls.append(len(text))
        
    avgdl = np.mean(dls)
    N = len(dls)
    
    return dls, avgdl, N

In [5]:
def preprocessing(query, stopwords):
    text = re.sub(r'[^\w\s]',' ',query).split()
    new_text = []
    for word in text:
        lemma = morph.parse(word)[0].normal_form
        if lemma not in stopwords:
            new_text.append(lemma)
    return ' '.join(new_text)

In [6]:
data = pd.read_csv('data.csv', sep='\t')

In [7]:
data = data.fillna('')

In [8]:
data.head()

,address,name,title,График работы,Опыт работы,Сфера деятельности,text
0,"Москва, м. Бульвар Дмитрия Донского, Старокача...",menedzher_-konsultant_951375511,Менеджер -консультант,полный день,не имеет значения,Продажи,Компании Мастер-трон требуется офис менеджер....
1,,ohrannik_v_tts_vodnyy_1201120983,Охранник в ТЦ Водный,сменный график,не имеет значения,"Охрана, безопасность",
2,"Москва, м. Дмитровская, Складочная улица, 1с13",uborka_ofisa_1535259521,Уборка офиса,,,,Требуется сотрудник для уборки офисного помеще...
3,"м. Охотный ряд, \n Москва",naborschik_zakazov_1502732417,Наборщик заказов,сменный график,более 1 года,"Транспорт, логистика",
4,"Москва, м. Нахимовский проспект, Нахимовский п...",voditel_taksi_1394522421,Водитель такси,сменный график,более 3 лет,Автомобильный бизнес,


In [9]:
corpus = []
russian_stopwords = set(stopwords.words('russian'))
for index, row in tqdm(data.iterrows()):
   # print(row)
    text = ' '.join(row[['title', 'text']])
    corpus.append(preprocessing(text, russian_stopwords))

A Jupyter Widget

In [10]:
with open('data_prep.txt', 'w', encoding='utf-8') as f:
    corpus_text = '\n'.join(corpus)
    f.write(corpus_text)

In [11]:
dls, avgdl, N = get_dls(corpus)

A Jupyter Widget

## ОБРАТНЫЙ ИНДЕКС

In [12]:
def create_matrix(corpus):
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(corpus)
    with open ('vectorizer', 'wb') as f:
        pickle.dump(vectorizer, f)
    
    vocabulary = vectorizer.vocabulary_.keys()
    print(len(vocabulary))
    matrix = pd.DataFrame(0, index=np.arange(N), columns=vocabulary)
    for word in tqdm(vocabulary):
        column = [len(re.findall(word, doc)) for doc in corpus]
        matrix[word] = column 

    return matrix, vocabulary

In [149]:
#vectorizer.idf_

array([ 6.62401751,  5.74854877,  7.54030824, ...,  7.13484313,
        7.54030824,  7.54030824])

In [13]:
matrix, vocabulary = create_matrix(corpus)

16533


A Jupyter Widget

In [22]:
matrix.head()

,менеджер,консультант,компания,мастер,трон,требоваться,офис,требование,грамотный,речь,...,петербургский,клубника,провозить,гортранспросервис,штурмовой,зажигаться,циклический,терпеливо,полно,дотошность
0,2,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
sparse_matrix = 0

In [24]:
sparse_matrix

0

In [18]:
scipy.sparse.save_npz('matrix.npz', sparse_matrix)

In [19]:
sparse_matrix = scipy.sparse.load_npz('matrix.npz')

In [20]:
new_matrix = pd.SparseDataFrame(sparse_matrix)

KeyboardInterrupt: 

In [25]:
matrix.to_pickle('matrix.pkl')

MemoryError: 

In [16]:
matrix.to_csv('matrix.csv', sep='\t', encoding='utf-8', index=False)

In [26]:
with open('vocabulary.txt', 'w', encoding='utf-8') as f:
    text = '\n'.join(vocabulary)
    f.write(text)

In [27]:
def score_BM25(qf, dl, avgdl, n, N) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    k1 = 2.0
    b = 0.75
    idf = math.log(1 + (N-n+0.5)/(n+0.5))
    score = idf * (k1+1)*qf/(qf+k1*(1-b+b*(dl/avgdl)))
    return score

In [28]:
def compute_sim(word, vocabulary, matrix, dls, avgdl, N) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    
    if word in vocabulary:
        n = len(matrix[matrix[word]>0])
        result = {}
        for i, t in enumerate(matrix[word]):
            qf = t/dls[i]
            #qf = term_doc_matrix[dictionary[word]][doc]
            
            score = score_BM25(qf, dls[i], avgdl, N, n)
            result[i] = score
        return result
    else:
        return {}

In [69]:
def okapi(q):
    new_q = preprocessing(q, russian_stopwords)
    print(new_q)
    
    res = defaultdict(int)
    for word in new_q.split():
        sim_dict = compute_sim(word, vocabulary, matrix, dls, avgdl, N)
        for k in sim_dict:
            res[k] += sim_dict[k]

    top = sorted(res, key=res.get, reverse=False)[:10]
    print(top)

    for el in top:
        print(el, res[el])
        print(data['title'][el] + data['text'][el])
   

In [70]:
okapi('бухгалтер женщина')

бухгалтер женщина


C:\Users\zhenya\A3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()


[3325, 3861, 1712, 7709, 771, 2262, 470, 7215, 9680, 3686]
3325 -2.51885508779
Бухгалтер
3861 -2.51885508779
Бухгалтер
1712 -1.0491068559
Бухгалтер по учету затрат
7709 -0.954879419037
Бухгалтер по учету имущества
771 -0.717204315772
Специалист по документам в бухгалтерию
2262 -0.623023919137
Главный бухгалтер в единственном лице
470 -0.604542880283
Упаковщики / Мужчины и Женщины на склад вахта
7215 -0.547973164718
Бухгалтер по строительно монтажным работам
9680 -0.253467647501
Товаровед- бухгалтерТребуется товаровед для учёта и ведения бухгалтерии 1С (розница) в  магазинах розничных продаж одежды и обуви..
3686 -0.126772128248
БухгалтерОписание работодателя:В небольшую строительную фирму требуется Бухгалтер с высшим профильным образованием
Требования:Опыт от 10 лет.
Обязанности: Ведение бухгалтерии, отчетность и т.д.
Условия:З/П и график обсуждается на собеседовании


In [72]:
data[470:471]

,address,name,title,График работы,Опыт работы,Сфера деятельности,text
470,,upakovschiki_muzhchiny_i_zhenschiny_na_sklad_v...,Упаковщики / Мужчины и Женщины на склад вахта,вахтовый метод,не имеет значения,"Без опыта, студенты",


## WORD2VEC

In [17]:
from gensim.models import Word2Vec, KeyedVectors

In [18]:
model = Word2Vec.load('araneum_none_fasttextskipgram_300_5_2018.model')

In [27]:
model.wv['менеджер']

array([ 0.08089462,  0.00701773, -0.00935735,  0.12002733, -0.31280148,
        0.0228413 , -0.01529199, -0.02186411, -0.0387328 ,  0.08120836,
       -0.09511425,  0.28745091,  0.08661713,  0.15903006,  0.08705243,
        0.02499125, -0.0635455 ,  0.15918148,  0.15837561,  0.01393018,
       -0.12084688,  0.08646525, -0.00407984, -0.1226891 ,  0.14332375,
        0.0383391 ,  0.15435955,  0.29652512, -0.17369893,  0.01833459,
       -0.07329725, -0.11599656, -0.10199031,  0.01565463,  0.16002776,
       -0.25694409, -0.09755956,  0.45247158, -0.24680077,  0.19071096,
       -0.07415186, -0.46609646, -0.27458704,  0.49468029, -0.07068588,
        0.00258506, -0.01159397,  0.16310877, -0.06888103, -0.13344339,
       -0.02761816,  0.1118577 , -0.15993267, -0.29807198, -0.22656329,
       -0.0712532 ,  0.21073416,  0.28499791, -0.08131388, -0.05281293,
        0.00485847,  0.27990091, -0.34847385, -0.1391923 , -0.37563425,
        0.53031272, -0.26867482,  0.10706437, -0.12842719, -0.16

In [48]:
def get_w2v_vectors(q):
    """Получает вектор документа"""
    new_q = preprocessing(q, russian_stopwords)
    vector = np.zeros(300)
    
    tfidf = {}
    for key, value in enumerate(vectorizer.transform([new_q]).toarray()[0]):
        if value != 0:
            tfidf[key] = value
    
    for word in new_q.split():
        try:
            vector += model.wv[word] * tfidf[vectorizer.vocabulary_[word]]
        except KeyError:
            pass
    if len(new_q) > 0:
        return (vector / len(new_q)).tolist()
    return np.zeros(300).tolist()
        

def save_w2v_base(corpus):
    """Индексирует всю базу для поиска через word2vec"""
    w2v_base = []
    for text in tqdm(corpus):
        vector = get_w2v_vectors(text)
        w2v_base.append(vector)
    return w2v_base

In [45]:
w2v_base[0:2]

[array([  9.24988817e-04,  -2.43970742e-03,  -2.02903102e-03,
         -1.03514835e-03,  -9.85405251e-04,  -2.35818753e-03,
         -5.76676105e-04,  -1.40639352e-03,   1.13961237e-03,
         -1.00858823e-03,  -1.36542582e-03,  -8.73678781e-04,
         -7.23900340e-04,  -2.64275364e-04,  -6.86114122e-04,
         -7.76953119e-04,   3.14956547e-04,   2.10097198e-03,
          2.19602776e-04,  -2.28190543e-03,  -1.63405162e-03,
         -4.31618405e-04,   1.27015824e-03,  -1.96312886e-03,
          3.61811976e-04,   1.19152157e-04,   9.36302677e-04,
          7.80963828e-04,  -1.01401274e-03,  -1.01266730e-03,
         -8.22583074e-04,  -1.41372507e-03,  -1.25981420e-03,
         -2.45743873e-04,   1.38395940e-03,  -4.85948608e-04,
         -2.62771495e-03,   4.15546177e-03,   4.73962031e-04,
          1.31547595e-03,   1.02564349e-03,  -2.69248630e-03,
         -1.62139236e-03,   2.90814463e-03,  -2.10766106e-03,
          5.59170993e-05,   9.37736119e-05,  -4.76548417e-04,
        

In [49]:
w2v_base = save_w2v_base(corpus)

A Jupyter Widget

In [50]:
with open('w2v_base.json','w') as f:
    json.dump(w2v_base, f, ensure_ascii=False)

In [139]:
def w2v_sim(q, w2v_base):
    vector = get_w2v_vectors(q)
    
    res = defaultdict(int)
    for i, base_vec in enumerate(w2v_base):
        #print(i, base_vec)
        res[i] = 1 - spatial.distance.cosine(vector, base_vec)

    top = sorted(res, key=res.get, reverse=True)[:10]
    print(top)

    for el in top:
        print(el, res[el])
        print(data['title'][el] + data['text'][el])
   

In [140]:
w2v_sim('бухгалтер без опыта', w2v_base)

C:\Users\zhenya\A3\lib\site-packages\scipy\spatial\distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


[5779, 8405, 2623, 853, 1122, 3686, 7215, 966, 5266, 2196]
5779 0.729568504796
Кассир-стажер (без опыта, выплаты еженедельно)
8405 0.721348033247
БухгалтерОписание работодателя:В Компанию требуется бухгалтер , с образованием (бухгалтерским ) Знание и умение работать на всех участках бухгалтерии. Опыт работы минимум 1 год. Желательно с знанием английского языка.
Требования: Строительная фирма "ВК строй".
5 дней работы в неделю.С 
Обязанности:Бухгалтерский учет,проверять  документы,подачи документов,встреча с другими компаниями,счета-фактуры,проверять соглашения
Условия:
2623 0.708425030814
Бухгалтер с опытом от 3 летОписание работодателя: Юридическая фирма РЕГИСТРАЛ, которая обслуживает компании различной формы собственности

Требования: высшее образование (бухгалтер, экономист), опыт от 3 лет

Обязанности: знание 1С, полное ведение бухгалтерского учёта по всем участкам (зарплата, касса, расчёты с поставщиками и другие), умение вести компании на УСН, ОСНО и специальных режимах налогообл

## DOC2VEC

In [155]:
documents = [TaggedDocument(doc, [i]) for i, doc in tqdm(enumerate(corpus))]
d2v_model = Doc2Vec(documents, vector_size=100, min_count=0, alpha=0.025, min_alpha=0.025, epochs=100)

A Jupyter Widget

In [156]:
d2v_model

In [157]:
def save_d2v_base(corpus):
    d2v_base = []
    for text in tqdm(corpus):
        vector = d2v_model.infer_vector(preprocessing_query(text))
        d2v_base.append(vector)
    return d2v_base
    

In [158]:
d2v_base = save_d2v_base(corpus)

A Jupyter Widget

In [163]:
def d2v_sim(q, d2v_base):
    vector = d2v_model.infer_vector(preprocessing_query(q))
    
    res = defaultdict(int)
    for i, base_vec in enumerate(d2v_base):
        #print(i, base_vec)
        res[i] = 1 - spatial.distance.cosine(vector, base_vec)

    top = sorted(res, key=res.get, reverse=True)[:10]
    print(top)

    for el in top:
        print(el, res[el])
        print(data['title'][el] + data['text'][el])

In [170]:
d2v_sim('ищу работу учителем', d2v_base)

[2833, 2391, 4069, 1370, 1149, 7860, 1439, 1664, 5674, 9198]
2833 0.323804793579
Кассир
2391 0.312569395603
ОтделочникиОписание работодателя: на отделку квартир требуются отделочники (совмещение приветствуется).
Требования: опыт работы 
Обязанности: выполнять работы качественно с соблюдением технологии и строительных норм
Условия: оплата сдельная
4069 0.311581894626
Ремонт собственной квартирыНужен ремонт собственной двухкомнатной квартиры под ключ. 
Качественно в срок. Площадь 67м2 вторичного дома.
Ответственность и исполнительность специалистов обязательно. 
Готов посмотреть ремонт на примере вашей работы.

Хорошая оплата труда.
1370 0.306709402728
Кассир (частичная занятость)
1149 0.303545667246
Требуется продавец в булочнуюОписание работодателя: ИП
Требования:
Обязанности:
Условия:
7860 0.294992261726
Охранник (г.Красногорск)
1439 0.288270634361
Продавец-консультант в салон сотовой связи Билайн
1664 0.280539892897
ОхранникОписание работодателя: В ООО ЧОО "Альфа-ПСК"требуются сотруд

In [35]:
with open ('vectorizer', 'rb') as f:
    vectorizer = pickle.load(f)